# Lorimier's Method

Below are the yields (in percentage points) on Swiss government bonds as of July 2015:
Construct the forward curve using Lorimier's method for $\alpha=0.1$. Report the yield on a zero-coupon bond with 6 years to maturity. Express your answer in percentage points and round to 2 decimal places.

In [1]:
dict_y = {2:-0.79, 3:-0.73, 4:-0.65, 5:-0.55, 7:-0.33, 10:-0.04, 20:0.54, 30:0.73}
for k, v in dict_y.items():
    dict_y[k] = v / 100
dict_y

{2: -0.0079,
 3: -0.0073,
 4: -0.006500000000000001,
 5: -0.0055000000000000005,
 7: -0.0033,
 10: -0.0004,
 20: 0.0054,
 30: 0.0073}

In [2]:
import numpy as np

def H_i(T_i, x):
    return T_i + T_i * min(T_i, x) - (min(T_i, x)) ** 2 / 2
def integral_1(T_i, u):
    return T_i * u + T_i * u**2 / 2 - u**3 / 6

def integral_2(T_i, x):
    return (T_i + T_i**2 / 2) * (x - T_i)

class Lorimier:
    def __init__(self, name): 
        self.T_list = list(dict_y.keys())
        self.y_list = list(dict_y.values())
        self.N = len(self.T_list)
        self.alpha = 0.1
        self.dict_product = self.solveProduct()
        self.beta = self.solveBeta()
    
    def solveProduct(self):
        N = self.N
        T_list = self.T_list
        d = {}
        for i in range(N):
            for j in range(i + 1):
                d[(i, j)] = -(min(T_list[i], T_list[j])) ** 3 / 6 + T_list[i] * T_list[j] * (1 + min(T_list[i], T_list[j]) / 2)
        return d
    
    def solveBeta(self):
        alpha = self.alpha
        N = self.N
        T_list = self.T_list
        
        A = np.zeros((N + 1, N + 1), dtype='float')
        b = np.zeros((N + 1,), dtype='float')
        A[0:1, 1:] = T_list
        
        for i in range(N):
            A[i + 1][0] = alpha * T_list[i] # beta_0
            for j in range(N):
                key = (i, j) if i > j else (j, i)
                A[i + 1][j + 1] = alpha * self.dict_product[key]
            A[i + 1][i + 1] += 1
            b[i + 1] = alpha * self.y_list[i] * self.T_list[i]
        print(b)
        return np.linalg.inv(A) @ b
    
    def f(self, x): # instantaneous forward rate
        res = self.beta[0]
        for i in range(1, len(self.beta)):
            res += self.beta[i] * H_i(self.T_list[i - 1], x)
        return res        
    
    def integral_1(T_i, u):
        return T_i * u + T_i * u**2 / 2 - u**3 / 6
    
    def integral_2(T_i, x):
        return (T_i + T_i**2 / 2) * (x - T_i)
        
    def __call__(self, x): # yield
        res = 0
        for i in range(self.N):
            T_i = self.T_list[i]
            if T_i < x:
                res += self.beta[i + 1] * (integral_1(T_i, T_i) + integral_2(T_i ,x))
            else:
                res += self.beta[i + 1] * (integral_1(T_i, x))
                
        return res / x + self.beta[0]

model = Lorimier(dict_y)

[ 0.      -0.00158 -0.00219 -0.0026  -0.00275 -0.00231 -0.0004   0.0108
  0.0219 ]


In [3]:
model(6) * 100

-0.41377721233020487